In [1]:
from tsfel.feature_extraction import features as tsfelf
import numpy as np
import os
from scipy import signal
import pandas as pd
import glob
import sklearn
from scipy.io import wavfile
from scipy.signal import decimate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
source_samp_rate=44100
decimation_factor = 4
#actual samples per second. When we record 1 second of audio we get only something like 0.9 seconds because of the chunk size
chunk=4096
single_audio_frame_size=10*chunk

In [3]:
def hpf(arr,fs,lf,order):
    wbut = lf
    bbut, abut = signal.butter(order,wbut, btype='highpass', fs=fs)
    return signal.filtfilt(bbut, abut, arr)

def get_features(sig,label,fs = 44100):
    
    s_f = hpf(sig, fs, 65, 2)

    Zero_Cross = tsfelf.zero_cross(sig)
    Zero_Cross_filt = tsfelf.zero_cross(s_f)
            
    Spectral_centroid = tsfelf.spectral_centroid(sig, fs)
    Spectral_centroid_filt = tsfelf.spectral_centroid(s_f, fs)

    Spectral_spread = tsfelf.spectral_spread(sig, fs)
    Spectral_spread_filt = tsfelf.spectral_spread(s_f, fs) 
            
    STD = np.std(sig)
    STD_filt = np.std(s_f)
    
    RMS = tsfelf.rms(sig)
    RMS_filt= tsfelf.rms(s_f)
    
    kurt_filt = tsfelf.kurtosis(s_f)
    
    return [label, RMS, RMS_filt, Zero_Cross, Zero_Cross_filt, Spectral_centroid, 
     Spectral_centroid_filt, Spectral_spread, Spectral_spread_filt, STD, STD_filt, kurt_filt]

In [4]:
def get_features_from_folder_3s_samples(folder_path,label,decimation_factor):
    files=glob.glob(folder_path+"/*.wav")
    #print(files)
    features_array = []
    for file_path in files:
        samplerate, sdata=wavfile.read(file_path)
        decimated_data=decimate(sdata,decimation_factor)
        sample1=decimated_data[0:int(single_audio_frame_size/decimation_factor*3)]
        print(len(sample1))
        features1=get_features(sample1,label,samplerate/decimation_factor)
        features_array.append(features1)
    return features_array

In [11]:
cwd = os.getcwd()
directory_contents = os.listdir(cwd)
list_of_directories = []

for item in directory_contents:
    if os.path.isdir(item):
        list_of_directories.append(item)

all_features=[]
for folder in list_of_directories:
    class_features = get_features_from_folder_3s_samples(cwd+"/"+folder,folder,decimation_factor)
    all_features=all_features+class_features

columnlist=["Label","RMS", "RMS_filt","Zero_Cross", "Zero_Cross_filt", "Spectral_centroid", "Spectral_centroid_filt", "Spectral_spread", \
                        "Spectral_spread_filt", "STD", "STD_filt", "kurt_filt"]
roadtype_features_df=pd.DataFrame(all_features,columns=columnlist)
filename="testset df-"+str(decimation_factor)+"_wl-3s_"+"26.2.2022.csv"
roadtype_features_df.to_csv(filename,index=False)
roadtype_features_df.head()





30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720


,Label,RMS,RMS_filt,Zero_Cross,Zero_Cross_filt,Spectral_centroid,Spectral_centroid_filt,Spectral_spread,Spectral_spread_filt,STD,STD_filt,kurt_filt
0,Snow,4075.916634,3516.219436,3241,3757,876.404458,947.739311,908.088175,897.468076,4075.916099,3516.218464,4.101836
1,Snow,3849.566540,3212.391466,3471,4169,901.628890,987.800372,895.958648,889.287051,3849.556770,3212.390946,5.317660
2,Snow,3747.516422,3072.274278,3058,3773,864.526699,961.664758,898.833970,906.627081,3747.515085,3072.273526,0.525560
3,Snow,4212.135860,3362.625534,2914,3721,835.997727,936.540447,892.564740,896.879875,4212.130097,3362.625425,2.363458
4,Snow,4323.637852,3219.383806,2776,3642,833.056621,945.401825,907.823572,899.762640,4323.629102,3219.382926,0.919938
